# Rasa Denerator

A simple way of generating a domain.yml file for Rasa

**Works with legacy versions of rasa || rasa 1.x!**

Developed with Python 3.7.x

## Installation

```bash
$ pip install rasa_denerator
```

After installation, the component can be used via command line or with a script, we provide scenarios for both.

## Usage

### CLI

The example below will create a domain file from an actions module, nlu training data and templates

```bash
$ python -m rasa-denerator -actions Rasa-Denerator/notebooks/denerator_tests/actions -f templates Rasa-Denerator/notebooks/denerator_tests/data -nlu Rasa-Denerator/notebooks/denerator_tests/data/nlu/nlu.md 
 ```
 
 ```bash
usage: rasa_denerator.py [-h] [-o OUTPUT] [-nlu NLU_FILE]
                         [-actions ACTIONS_DIR] [-f TAG_FILES TAG_FILES]

merge disparate Rasa domain files to create or update a aggregated domain.yml

optional arguments:
  -h, --help            show this help message and exit
  -o OUTPUT, --output OUTPUT
                        -o <PATH_TO_OUTPUT_FILE>
  -nlu NLU_FILE, --nlu_file NLU_FILE
                        -nlu <PATH_TO_NLU_TRAINING_DATA>
  -actions ACTIONS_DIR, --actions_dir ACTIONS_DIR
                        -actions <PATH_TO_ACTIONS_DIRECTORY>
  -f TAG_FILES TAG_FILES, --find TAG_FILES TAG_FILES
                        -f <TAG> <PATH_TO_DIRECTORY_OR_FILE>
 ```

The script will output to the std.out unless an output file is specific. 

Queries can get quite complex. For example, you could specify the following:
```bash
$ python3 -m rasa_denerator -actions Rasa-Denerator/notebooks/denerator_tests/actions \ 
         -f templates Rasa-Denerator/notebooks/denerator_tests/data \
         -f slots Rasa-Denerator/notebooks/denerator_tests/data/ \
         -nlu Rasa-Denerator/notebooks/denerator_tests/data/nlu/nlu.md \
         -o domain.yml
 ```

### Python

If you'd like to use the denerator within a python script, you can do that too!
```python
from rasa_denerator import RasaDenerator

nlu_file = "denerator_tests/data/nlu/nlu.md"
actions_dir = "denerator_tests/actions/"
tag_dict = {"templates":"denerator_tests/data/domain", "slots": "denerator_tests/data/domain", "entities":"denerator_tests/data/domain"}
output_file = "domain.yml"

denerator = RasaDenerator(nlu_file = nlu_file, 
                        actions_dir = actions_dir,
                        tag_dict = tag_dict,
                        output = output_file)
denerator.generate_domain()

```

## Explanation

Creating a domain.yml file currently is a tedious process within Rasa.  Take this example below

Here is a sample structure that your rasa project might look like right now.

```
your_rasa_project
│   README.md
│   other stuff...   
│
└───data (This is where you store all your data)
│   nlu.md
|   domain.yml
|       
└───actions (This is where your action.py lives for rasa_sdk)
    │   __init__.py
    │   actions.py
    |   other_code.py
```

Currently the developer must hand create the domain file using multiple assets from other files that already exists within other parts of the application. Here are some points below
 
1. Entities and intents that are placed within training data files must be copied manually to a domain.yml file prior to training. 
2. Custom action names must be listed within the domain.yml file. Developers currently need to manually extract these names from their custom action classes everytime they add a new actions prior to retraining
3. Templates are defined within the domain.yml file. These templates are utterances that can be returned to the user at certain trained points within the dialogue. Once these template utterances are defined, their identifies must be manually copied to the actions section of the domain.yml file.

You might have felt these pain points before...

To alleviate some pain, I created this tool to automatically generate a domain.yml file

### Dev considerations: 
- Automatically generate intents based on intents listed within NLU training data
- Automatically generate actions based on a valid actions module  
- Automatically aggergate forms, actions, slots, utterances listed within a specific directory recursivly
- If NLU training data exists or Actions module exists, tags read in for entities, intents, actions, or forms are ignored
- Make it command line version accessable
- Make it easy to include within another script

### A Better Layout

Instead of hand creating these domain files and updating them constantly as you create new entities, intents, utterances, and actions consider the layout below. 

```
your_rasa_project
│   README.md
│   other stuff...   
│
└───data (This is where you store all your data)
│   │
│   └───nlu
│       │   nlu.md
|       |
│   └───domain
│       │   templates_1.yml (file containing templates)
|       |   templates_2.yml (file containing more templates...)
│       │   slots_1.yml (file containing slots)
|       |   slots_2.yml (file containing more slots...)
│       │   entities.yml (file containing entities, nlu training entities take precedence over this file)
│       │   intents.yml (file containing intents, nlu training intents take precedence over this file)
|       
└───actions (This is where your action.py lives for rasa_sdk)
    │   __init__.py
    │   actions.py
    |   other_code.py
```

#### Utterances

All utterance templates can be defined within a set of yml files. The script will automatically extract these templates and add them to a domain file as templates and registed actions. Additionally, the script will append these to registered utterances to the registered actions within the generated domain file. 

#### Entities and Intents
Entities and intents will be extracted from the nlu.md training data file. This script uses Rasa's training_data functions to load and extract entities and intents. This allows us to accept markdown, json, etc. Entities and intents within the training data are extracted and placed into the output file.

If you want to hand-define entities and intents  thats fine too. Just create a folder or file that contains them and pass it to the script. 

Note: If a entities or intents are specified, they will take precidence over entities and intents identified within the nlu training data file.

#### Actions
Most Rasa users define custom actions. Copying these names from their respective classes and copying them into the domain file is currently tedious. The denerator fixes this my using rasa_sdk to  load the repsective action module created by the user, extract the class names, and automatically add them to the domain file.

## Tests

All unit tests are within the notebook file and can be run here. I prototyped all code within the notebook, all unit tests can be found there.

## License 

MIT 2019

## Questions

Contact me at collen.roller@gmail.com

In [25]:
from rasa.nlu.training_data import loading

In [40]:
"""
rasa_denerator.py
August 10, 2019
collen.roller@gmail.com

Domain aggergator to merge disparate files to create a aggregated domain.yml file for Rasa 

See README.md for details

"""

from argparse import ArgumentParser
import ruamel.yaml as yaml
from ruamel.yaml import YAML
import os
import os.path
import sys
import logging
from os import listdir, remove, removedirs, makedirs
from os.path import isfile, join, exists
from typing import Any, Optional, Text, Dict, Set, List
from rasa.nlu.training_data import loading
from rasa.nlu.training_data import Message, TrainingData
from rasa_sdk.executor import ActionExecutor
from pathlib import Path 

VALID_SEARCH_TAGS = ["forms", "actions", "templates", "slots", "entities", "intents"]

logger = logging.getLogger(__name__)

class RasaDenerator:
    
    def __init__(self,
                 nlu_file: Optional[Text] = None,
                 actions_dir: Optional[Text] = None,
                 tag_dict: Optional[Dict] = None,
                 output: Optional[Text] = None
                ) -> None:
        
        self.nlu_file = nlu_file
        self.actions_dir = actions_dir

        if type(tag_dict) == list:
            tag_dict = self.convert_tags(tag_dict)
        
        self.tag_dict = tag_dict
        self.output = output
           
    def generate_domain(self) -> Text:
    
        logger.debug("Generating domain file")
        results = self.get_tagged_entries(self.tag_dict) # Get tagged entities
        
        # if nlu_file is specified, look for intents and entities within the nlu file. 
        if self.nlu_file:
            logger.debug("Extracting entities and intents from nlu training data (%s)" % self.nlu_file)
            nlu_data = loading.load_data(self.nlu_file)
            if nlu_data:
                if len(results["entities"]) == 0:
                    results["entities"] = list(nlu_data.entities)
                if len(results["intents"]) == 0:
                    results["intents"] = list(nlu_data.intents)
                    
        # if actions_file is specified, look for registed actions within actions file.
        # Keep only the actions / forms that were found within the NLU file
        if self.actions_dir:
            logger.debug("Extracting actions from action directory (%s)" % self.actions_dir)
            actions = self.get_actions(self.actions_dir)
            if "actions" in actions.keys(): results["actions"] = actions["actions"]
            if "forms" in actions.keys(): results["forms"] = actions["forms"]
         
        logger.debug("Merging identified utterances")
        # If templates exist, append them to the actions
        if "templates" in results.keys() and len(results["templates"]) > 0:
            results["actions"] = results["actions"] + list(results["templates"].keys())       
        
        logger.debug("Formatting output")
        # Iterate through output, identify existing tags, and remove keys that dont exist
        for tag in VALID_SEARCH_TAGS:
            if tag in results.keys() and len(results[tag]) > 0:
                print("Found %s %s" % (len(results[tag]), tag))
            else:
                # remove the keys from the list
                del results[tag]
                logger.warning("No %s found" % tag)  
    
        # output the results to std out, if an output file was specified, send it there
        yaml = YAML()
        yaml.compact(seq_seq=False, seq_map=False)
        if self.output:
            #output to file
            if os.path.isdir(self.output): 
                logger.error("Output location (%s) is a directory.. can not overwrite" % self.output)
                return "Output location (%s) is a directory.. can not overwrite" % self.output
            elif os.path.isfile(self.output):
                logger.warning("Output file %s already exists, overwritting..." % self.output)
                
            output_path = self.output if os.path.isabs(self.output) else os.path.join(os.path.abspath(os.curdir), self.output)
            
            try:
                stream = open(output_path, "w")
            except IOError:
                stream = open(Path(output_path), "w") #Create the file!
                
            yaml.dump(results, stream)
            print("Results saved to %s" % self.output)
        else:    
            # yaml.dump(results, sys.stdout)
            yaml.dump(results, sys.stdout)
            
            
    # Iterates through use specified files and returns tagged information within yml files
    def get_tagged_entries(self, tag_dict):
        
        results = {}
        # Set results for key to an empty list
        for tag in VALID_SEARCH_TAGS:  results[tag] = []

        # iterate through tag_dict and recursivly look through directories for .yml files with tags.
        for key, path in tag_dict.items():
            
            if os.path.isdir(path):  
                # Recursivly find all .yml files within sub directories
                canidates = [os.path.join(subdir, file) 
                             for subdir, dirs, files in os.walk(path) 
                             for file in files if file.endswith(".yml")]
                
                # Read YML Files for key
                for candidate in canidates:
                    # Go through each file and look for elements to add to the results for the key
                    yml = self.get_yml(key, candidate)
                
                    if yml and key in yml: 
                        # if templates, dive down another level
                        if len(results[key]) > 0 and "templates" in key:
                            results[key][0].update(yml[key])
                        else:
                            results[key].append(yml[key])
            
            elif os.path.isfile(path) and path.endswith(".yml"):  
                yml = self.get_yml(key, path)
                if yml and key in yml:
                    results[key].append(yml[key])
                    
                    
        # Once complete, go through each key
        for key, value in results.items():
            # Merge content that exists!
            if len(results[key]) > 0 and type(results[key][0]) is dict:
                results[key] = {k: v for d in results[key] for k, v in d.items()}
            elif len(results[key]) > 0 and type(results[key][0]) is list:
                results[key] = [item for i in results[key] for item in i]

        return results
    
    # Provide the actions directory that will be mounted for production
    # i.e. actions/actions.py
    #      actions/__init__.py
    # Pass <PATH_TO_ACTIONS_DIRECTORY> 
    def get_actions(self, location):
        if os.path.isdir(location):  
            sys.path.insert(0, location)
            a_exec = ActionExecutor()
            
            # os.path.basename did not work here for some reason unknown.
            a_exec.register_package(Path(location).name)
            actions = list(a_exec.actions.keys())
            # Return a dictionary of actions and forms
            # NOTE: FORM NEEDS TO HAVE FORM IN NAME, ACTIONS NEED ACTION IN NAME
            return {"actions": [item for item in actions if item.startswith("action")], 
                    "forms": [item for item in actions if item.startswith("form")]}
        elif os.path.isfile(location): 
            logger.error("%s must be a model directory, not a file" % location)

        return None
    
    
    # Create class structure 
    def get_yml(self, key, file) -> Optional[Dict]:
        
        try:
            loaded_yml = yaml.safe_load(open(file, "r"))
            if loaded_yml and key in loaded_yml:
                return {key: loaded_yml[key]}
        except yaml.YAMLError as exc:
            logger.error("Error loading yml file %s" % file)
            logger.error(exc)
        return None 
    
    @staticmethod
    def create_argument_parser() -> ArgumentParser:
        parser = ArgumentParser(
            description='merge disparate Rasa domain files to create or update a aggregated domain.yml')
        parser.add_argument('-o', '--output',
                           required=False,
                           help='-o <PATH_TO_OUTPUT_FILE>')

        parser.add_argument('-nlu', '--nlu_file',
                            required=False,
                            help='-nlu <PATH_TO_NLU_TRAINING_DATA>')

        parser.add_argument('-actions', '--actions_dir',
                            required=False,
                            help='-actions <PATH_TO_ACTIONS_DIRECTORY>')
        
        parser.add_argument('-f', '--find',
                            nargs=2,
                            action='append',
                            dest="tag_files",
                            required=False,
                            help='-f <TAG> <PATH_TO_DIRECTORY_OR_FILE>')  
        return parser

    @staticmethod
    def convert_tags(tag_list):
        
        tag_dict = {}
        for pair in tag_list: 
            if pair[0] not in ["forms", "actions", "templates", "slots", "entities", "intents"]:
                print("%s is not a valid tag [%s] ignoring" % pair[0])
            else:
                tag_dict[pair[0]] = pair[1] 

        return tag_dict

## Testing

- Eusure you have the library installed for this
- https://github.com/JoaoFelipe/ipython-unittest
- pip install ipython_unittest
- pip install jupyter_dojo

In [12]:
# Testing Main

In [28]:
# Load the module
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


In [27]:
# Import Libraries for Testing
# from rasa_denerator import RasaDenerator
from unittest import TestCase

In [29]:
%%unittest_testcase

# from IPython.core.debugger import Tracer; Tracer()() 

class RasaDeneratorTest(TestCase):
    
    # Testing creating arguments
    def test_create_args(self):
        print("test_create_args running...")
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        
        parsed = parser.parse_args("-nlu data/nlu/nlu.md \
                            -actions agents/v0/actions/ \
                            -f templates data/domains/ \
                            -f slots data/domains/ \
                            -o domain.yml".split())
        
        
        # Testing Data Structures
        self.assertEqual(parsed.nlu_file, 'data/nlu/nlu.md')
        
        # Testing Actions Directory
        self.assertEqual(parsed.actions_dir, 'agents/v0/actions/')
        
        # Testing Tags (Proper)
        tag_dict = {}
        for pair in parsed.tag_files: 
            tag_dict[pair[0]] = pair[1]
            self.assertEqual(tag_dict[pair[0]], pair[1])
        
        # Testing Output
        self.assertEqual(parsed.output, 'domain.yml')
    
    # Testing class initialization
    def test_set_up(self):
        print("test_set_up running...")
        
        # Test case 1... all parameters are present (Pass)
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        parsed = parser.parse_args("-nlu data/nlu/nlu.md \
                    -actions agents/v0/actions/ \
                    -f templates data/domains/templates \
                    -f slots data/domains/slots \
                    -o domain.yml".split())
            
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output)         
        self.assertEqual(denerator.nlu_file, "data/nlu/nlu.md")
        self.assertEqual(denerator.actions_dir, "agents/v0/actions/")
        self.assertDictContainsSubset(denerator.tag_dict, {"slots":"data/domains/slots", "templates":"data/domains/templates"})
        self.assertEqual(denerator.output, "domain.yml")
        
        # Test case 2... only output file exists (Problem)
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        parsed = parser.parse_args("-o domain.yml".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output)
        
        # Test case 3... no arguments (Problem)
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        parsed = parser.parse_args("".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output)
        
    # Testing class initialization
    def test_get_tagged_entries(self):
        print("test_get_tagged_entries running...")
        
        # Test case 1... empty directories specified..
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        parsed = parser.parse_args("-nlu data/nlu/nlu.md \
                    -actions actions/ \
                    -f templates data/domain/templates \
                    -f slots data/domain/slots \
                    -o domain.yml".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
            
        entries = denerator.get_tagged_entries(denerator.tag_dict)
        self.assertEqual(entries["templates"], [])
        self.assertEqual(entries["slots"], [])
        
        
        # Test case 2... directory has tags in it for slots and templates..
        parsed = parser.parse_args("-nlu denerator_tests/data/nlu/nlu.md \
                    -actions denerator_tests/actions/ \
                    -f templates denerator_tests/data/domain \
                    -f slots denerator_tests/data/domain \
                    -o denerator_tests/data/domain.yml".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
         
        entries = denerator.get_tagged_entries(denerator.tag_dict)
        self.assertGreaterEqual(len(entries["templates"]), 1)
        self.assertGreaterEqual(len(entries["slots"]), 1)
        
    def test_get_actions(self):
        
        # Test case 1... empty directories specified..
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        parsed = parser.parse_args("-nlu data/nlu/nlu.md \
                    -actions actions/".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
        actions = denerator.get_actions(denerator.actions_dir)
        #from IPython.core.debugger import Tracer; Tracer()() 
     
        # Test case 2... actions directory is specified!
        parsed = parser.parse_args("-nlu data/nlu/nlu.md \
                    -actions denerator_tests/actions/".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
        actions = denerator.get_actions(denerator.actions_dir)
        #from IPython.core.debugger import Tracer; Tracer()()
        
        # Test case 3... actions file is specified!
#         parsed = parser.parse_args("-nlu data/nlu/nlu.md \
#                     -actions denerator_tests/actions/actions.py".split())
#         denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
#         actions = denerator.get_actions(denerator.actions_dir)
#         #from IPython.core.debugger import Tracer; Tracer()()
 
    def test_generate_domain(self):
        
#         # Test case 1... nlu and actions..
#         parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
#         parsed = parser.parse_args("-nlu denerator_tests/data/nlu/nlu.md \
#                     -actions denerator_tests/actions/".split())
#         denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
#         denerator.generate_domain()
        
#         # Test case 2... nlu and actions and templates
#         parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
#         parsed = parser.parse_args("-nlu denerator_tests/data/nlu/nlu.md \
#                     -actions denerator_tests/actions/ \
#                     -f templates denerator_tests/data/domain/".split())
#         denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
#         denerator.generate_domain()
        
        # Test case 3... nlu, actions, templates with entities (entity file should take precident over nlu training data)
        parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
        parsed = parser.parse_args("-nlu denerator_tests/data/nlu/nlu.md \
                    -actions denerator_tests/actions/ \
                    -f templates denerator_tests/data/domain/ \
                    -f entities denerator_tests/data/domain/".split())
        denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
        denerator.generate_domain()
        
        
        
        # For some reason, output test arent working correctly
        # Test case 3... nlu and actions and templates and output file (rel path)
#         parser = RasaDenerator.create_argument_parser() # Test Extracting Arg Parser
#         parsed = parser.parse_args("-nlu denerator_tests/data/nlu/nlu.md \
#                     -actions denerator_tests/actions/ \
#                     -f templates denerator_tests/data/domain/ \
#                     -o denerator_test/domain_testout.yml".split())
#         denerator = RasaDenerator(parsed.nlu_file, parsed.actions_dir, parsed.tag_files, parsed.output) 
#         denerator.generate_domain()

test_create_args running...
test_get_tagged_entries running...
test_set_up running...


Fail

.E.EE
ERROR: test_generate_domain (__main__.RasaDeneratorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 139, in test_generate_domain
  File "/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/rasa_denerator-1.0.4-py3.7.egg/rasa_denerator/rasa_denerator.py", line 48, in generate_domain
    results = self.get_tagged_entries(self.tag_dict) # Get tagged entities
  File "/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/rasa_denerator-1.0.4-py3.7.egg/rasa_denerator/rasa_denerator.py", line 116, in get_tagged_entries
    for key,path in tag_dict.items():
AttributeError: 'list' object has no attribute 'items'

ERROR: test_get_tagged_entries (__main__.RasaDeneratorTest)
----------------------------------------------------------------------
Traceback (most recent call last):
  File "Cell Tests", line 74, in test_get_tagged_entries
  File "/home/gguy/code/dene

<unittest.runner.TextTestResult run=5 errors=3 failures=0>

In [17]:
# Importing the object
from rasa_denerator import RasaDenerator

# args = RasaDenerator.create_argument_parser().parse_args(
#                             "-nlu denerator_tests/data/nlu/nlu.md \
#                             -actions denerator_tests/actions/ \
#                             -f templates denerator_tests/data/domain \
#                             -f slots denerator_tests/data/domain \
#                             -f entities denerator_tests/data/domain".split())

nlu_file = "denerator_tests/data/nlu/nlu.md"
actions_dir = "denerator_tests/actions/"
tag_dict = {"templates":"denerator_tests/data/domain", "slots": "denerator_tests/data/domain", "entities":"denerator_tests/data/domain"}

denerator = RasaDenerator(nlu_file = nlu_file, 
                        actions_dir = actions_dir,
                        tag_dict = tag_dict)
denerator.generate_domain()

No forms found


Found 177 actions
Found 163 templates
Found 24 slots
Found 5 entities
Found 51 intents
actions:
- action_explain_sales_form
- action_chitchat
- action_faqs
- action_pause
- action_store_unknown_product
- action_store_unknown_nlu_part
- action_store_bot_language
- action_store_entity_extractor
- action_set_onboarding
- action_select_installation_command
- action_store_problem_description
- action_greet_user
- action_default_fallback
- action_next_step
- utter_ask_budget
- utter_ask_business_email
- utter_ask_company
- utter_great
- utter_ask_email
- utter_response_why_email
- utter_awesome
- utter_can_do
- utter_ask_goal
- utter_ask_job_function
- utter_ask_use_case
- utter_ask_continue_newsletter
- utter_ask_continue_sales
- utter_confirm_salesrequest
- utter_salesrequest_failed
- utter_confirmationemail
- utter_docu
- utter_quickstart_nlu_only
- utter_ask_migration
- utter_switch_dialogflow
- utter_switch_luis
- utter_greet
- utter_greet_name
- utter_greet_noname
- utter_inform_privac

    text: I can't sign you up if you don't provide your email 😉
  utter_ask_isbot:
  -
    text: Yep, I'm a bot!
  utter_cant_answer_tech_question:
  -
    text: I can't answer that.
  utter_no_email:
  -
    text: Hmm, I'm not sure that's a valid email, please make sure to include the
      full address 😅
  utter_have_you_used_rasa_before:
  -
    text: Have you used Rasa before?
  utter_built_bot_before:
  -
    text: Alright, have you built a bot before?
  utter_anything_else:
  -
    text: Is there anything else I can help you with?
  utter_encourage_building_bot:
  -
    text: Cool, then good luck with building your first "I'm the Rasa bot 🤖!
  utter_chatbot_tutorial:
  -
    text: "No worries. There's a course on Datacamp that teaches you how to build\
      \ your own chatbots: https://www.datacamp.com/courses/building-chatbots-in-python\
      \ \U0001F913"
  utter_no_guide_for_switch:
  -
    text: Sorry, but we don't have a guide for this yet.
  utter_ask_which_product:
  -
 

    text: 'The supported Python versions are: 2.7, 3.5, 3.6 🐍 The recommended version
      is 3.6.'
  utter_ask_faq_differencecorenlu:
  -
    text: Rasa NLU’s job is to interpret messages, and Rasa Core’s job is to decide
      what should happen next.
  utter_ask_faq_community_size:
  -
    text: Rasa has more than 3500 community members and over 200 github contributors
      🚀.
  utter_ask_faq_what_is_forum:
  -
    text: Our [forum](https://forum.rasa.com) is the place where all the community
      buzz happens. Join discussions, get the answers to your technical questions
      and stay up-to-date with what’s happening at Rasa.
  utter_ask_voice:
  -
    text: Yes, that is possible! You can connect Rasa to any channel you like. You
      can find more information [here](https://rasa.com/docs/core/connectors/).
  utter_faq_platform_more:
  -
    text: Please read our [documentation](https://rasa.com/docs/platform/) for more
      information.
  utter_faq_language_more:
  -
    tex

In [30]:
parser = RasaDenerator.create_argument_parser()
parser.parse_args("-nlu denerator_tests/data/nlu/nlu.md \
                    -actions denerator_tests/actions/ \
                    -f templates denerator_tests/data/domain \
                    -f slots denerator_tests/data/domain \
                    -o denerator_tests/data/domain.yml".split())

Namespace(actions_dir='denerator_tests/actions/', nlu_file='denerator_tests/data/nlu/nlu.md', output='denerator_tests/data/domain.yml', tag_files=[['templates', 'denerator_tests/data/domain'], ['slots', 'denerator_tests/data/domain']])

In [41]:
import unittest
if __name__ == '__main__':
    unittest.main(argv=['first-arg-is-ignored'], verbosity=2, exit=False)

test_create_args (__main__.RasaDeneratorTest) ... ok
test_generate_domain (__main__.RasaDeneratorTest) ... 

test_create_args running...


/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/ruamel/yaml/scanner.py:350: ResourceWarning: unclosed file <_io.FileIO name='denerator_tests/data/domain/slots.yml' mode='rb' closefd=True>
  for level in list(self.possible_simple_keys):
/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/ruamel/yaml/constructor.py:167: ResourceWarning: unclosed file <_io.TextIOWrapper name='denerator_tests/data/domain/templates.yml' mode='r' encoding='UTF-8'>
  data = constructor(self, node)
/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/ruamel/yaml/constructor.py:167: ResourceWarning: unclosed file <_io.TextIOWrapper name='denerator_tests/data/domain/templates_1.yml' mode='r' encoding='UTF-8'>
  data = constructor(self, node)
/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/ruamel/yaml/constructor.py:167: ResourceWarning: unclosed file <_io.FileIO name='denerator_tests/data/domain/entities.yml' mode='rb'

Found 177 actions
Found 163 templates
Found 5 entities
Found 51 intents
actions:
- action_explain_sales_form
- action_chitchat
- action_faqs
- action_pause
- action_store_unknown_product
- action_store_unknown_nlu_part
- action_store_bot_language
- action_store_entity_extractor
- action_set_onboarding
- action_select_installation_command
- action_store_problem_description
- action_greet_user
- action_default_fallback
- action_next_step
- utter_ask_budget
- utter_ask_business_email
- utter_ask_company
- utter_great
- utter_ask_email
- utter_response_why_email
- utter_awesome
- utter_can_do
- utter_ask_goal
- utter_ask_job_function
- utter_ask_use_case
- utter_ask_continue_newsletter
- utter_ask_continue_sales
- utter_confirm_salesrequest
- utter_salesrequest_failed
- utter_confirmationemail
- utter_docu
- utter_quickstart_nlu_only
- utter_ask_migration
- utter_switch_dialogflow
- utter_switch_luis
- utter_greet
- utter_greet_name
- utter_greet_noname
- utter_inform_privacypolicy
- utter

    text: I can spell baguette in French, but unfortunately English is the only language
      I can answer you in.
  -
    text: I am in the process of learning, but at the moment I can only speak English.
  utter_ask_restaurant:
  -
    text: I'm sorry, I can’t recommend you a restaurant as I usually cook at home.
  utter_ask_time:
  -
    text: It's the most wonderful time of the year!
  utter_ask_wherefrom:
  -
    text: I was born in Berlin, but I consider myself a citizen of the world.
  -
    text: I was born in the coolest city on Earth.
  utter_ask_whoami:
  -
    text: I hope you are being yourself.
  utter_handleinsult:
  -
    text: That’s not very nice 😢
  utter_nicetomeeyou:
  -
    text: Thank you. It is a pleasure to meet you as well!
  -
    text: It is nice to meet you too!
  -
    text: Pleased to meet you too!
  -
    text: Likewise!
  utter_telljoke:
  -
    text: Why are eggs not very much into jokes? - Because they could crack up.
  -
    text: Do you know a tree

ok
test_get_actions (__main__.RasaDeneratorTest) ... ok
test_get_tagged_entries (__main__.RasaDeneratorTest) ... 

test_get_tagged_entries running...


/home/gguy/code/denerator/Rasa-Denerator/venv/lib/python3.7/site-packages/ruamel/yaml/reader.py:185: ResourceWarning: unclosed file <_io.FileIO name='denerator_tests/data/domain/slots.yml' mode='rb' closefd=True>
  return FileMark(self.name, self.index, self.line, self.column)
ok
test_set_up (__main__.RasaDeneratorTest) ... 

test_set_up running...


/usr/lib/python3.7/unittest/case.py:1143: DeprecationWarning: assertDictContainsSubset is deprecated
  DeprecationWarning)
ok

----------------------------------------------------------------------
Ran 5 tests in 2.582s

OK
